# Semana 2 Laboratorio: Ingesta en Streaming

En este laboratorio, interactuarás con Amazon Kinesis Data Streams y obtendrás una mejor comprensión de cómo se realiza el proceso de ingesta en streaming. El laboratorio consta de dos partes:
1. En la primera parte, trabajarás con un Kinesis Data Stream que actúa como un enrutador entre un productor simple y un consumidor simple. Usando la aplicación del productor, generarás manualmente datos y los escribirás en el Kinesis Data Stream. Después, consumirás los datos generados desde ese stream.
2. En la segunda parte, realizarás un proceso ETL en streaming: consumirás datos de un Kinesis Data Stream que es alimentado por un productor. Aplicarás algunas transformaciones simples a estos datos, y luego pondrás los datos transformados en uno de los otros dos streams de datos. Desde cada uno de estos dos nuevos streams de datos, los datos serán tomados por un Kinesis Firehose y entregados a sus respectivos buckets de S3.

Para abrir los archivos de la solución, sigue estos pasos:
- Ve al menú principal y selecciona `Archivo -> Preferencias -> Configuración`.
- Haz clic en `Editor de texto` a la izquierda, luego desplázate hacia abajo hasta la sección `Archivos: Excluir`.
- Elimina las líneas `"**/src_solution/**"` y `**/C2_W2_Lab_1_Streaming_Ingestion_Solution.ipynb`. Los archivos ahora aparecerán en el explorador.
- Puedes cerrar la pestaña `Configuración`.

Comencemos con el laboratorio.


# Tabla de Contenidos
- [Introducción](#0)
- [ 1 - Entendiendo los Componentes de una Plataforma de Transmisión de Eventos](#1)
- [ 2 - Implementando un Proceso de ETL en Tiempo Real](#2)
  - [ 2.1 - Creando la Infraestructura para el Proceso de ETL en Tiempo Real](#2-1)
  - [ 2.2 - Implementando el ETL en Tiempo Real](#2-2)


<a id='0'></a>
## Introducción al Laboratorio

En el laboratorio de la Semana 4 del Curso 1, interactuaste con un Flujo de Datos de Kinesis que transmite en línea actividades de usuarios como eventos o registros. Luego procesaste estos registros para calcular las recomendaciones de productos y utilizaste una instancia de Data Firehose para entregar los registros a un bucket de S3 en tu canal de datos. En ese laboratorio, no te mostramos cómo puedes transmitir las recomendaciones de vuelta a la web. Entonces, en la segunda parte de este laboratorio, volverás a interactuar con el mismo Flujo de Datos de Kinesis como tu sistema fuente, ingestando desde él la actividad de usuarios en línea, realizando algunos cálculos en estos datos y finalmente creando tus propios Flujos de Datos de Kinesis para continuar transmitiendo los datos transformados dentro de tu canal de datos. En este laboratorio, para mantener simple el paso de transformación, no calcularás las recomendaciones de productos a partir de los datos ingeridos, en su lugar aplicarás algunas transformaciones simples a los datos ingeridos. Pero antes de realizar esos pasos (segunda parte del laboratorio), primero pasarás por la primera parte del laboratorio para obtener una mejor comprensión de los componentes de una plataforma de transmisión.


<a id='1'></a>
## 1 - Comprendiendo los Componentes de una Plataforma de Transmisión de Eventos

Una arquitectura orientada a eventos consiste en un productor, un enrutador (buffer/corredor de mensajes) y un consumidor. En esta primera parte del laboratorio, trabajarás con estos tres componentes:

<div style="text-align:center"><img src="images/Part1.png"  width="400"/></div>

- enrutador: crearás un Flujo de Datos Kinesis que actuará como enrutador entre un productor y un consumidor;
- productor: en la carpeta `src/cli`, puedes encontrar el script de Python: `producer_from_cli.py`. Este script contiene código que escribe un registro en un Flujo de Datos Kinesis. Llamarás a este script desde una interfaz de línea de comandos (CLI) y pasarás dos argumentos: el nombre del Flujo de Datos Kinesis y una cadena JSON que representa el registro del mensaje.
- consumidor: en la misma carpeta `src/cli`, puedes encontrar el script de Python: `consumer_from_cli.py` que también llamarás desde la interfaz de línea de comandos (CLI). Toma un argumento que es el nombre del Flujo de Datos Kinesis del cual el consumidor leerá los registros de mensajes.


Importar los paquetes necesarios:


In [ ]:
import subprocess

from IPython.display import HTML

1.1. Ejecuta el siguiente código para obtener el enlace a la consola de AWS.

*Nota*: Por razones de seguridad, la URL para acceder a la consola de AWS expirará cada 15 minutos, pero cualquier recurso de AWS que hayas creado permanecerá disponible durante el período de 2 horas. Si necesitas acceder a la consola después de 15 minutos, por favor vuelve a ejecutar esta celda de código para obtener un nuevo enlace activo.


In [ ]:
with open('../.aws/aws_console_url', 'r') as file:
    aws_url = file.read().strip()

HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')

*Nota:* Si ves la ventana como en la siguiente captura de pantalla, haz clic en el enlace **cerrar sesión**, cierra la ventana y vuelve a hacer clic en el enlace de consola.

![AWSLogout](images/AWSLogout.png)


1.2. En la consola de AWS busca **Kinesis**, y haz clic en **Crear flujo de datos**. Nómbralo `de-c2w2lab1-kinesis-data-stream-cli` y deja el resto como predeterminado. Haz clic en el botón **Crear flujo de datos**. Una vez que esté en estado `Activo` puedes continuar con los siguientes pasos.

<img src="images/CreateDataStream.png"  width="300"/>


1.3. En la terminal, ejecuta los siguientes comandos para navegar a la carpeta `src/cli` y ejecutar el script de Python `consumer_from_cli.py`. El script del consumidor espera un argumento (el nombre del flujo de datos) que se especifica usando la bandera `--stream` seguida del nombre del flujo de datos:

```bash
cd src/cli/
python consumer_from_cli.py --stream de-c2w2lab1-kinesis-data-stream-cli
```

El consumidor comenzará a iterar por todos los fragmentos del flujo de datos, leyendo todos los registros de cada fragmento y mostrando alguna información sobre cada registro en la terminal. Sin embargo, ahora no aparecerá nada incluso si esperas 1 o 2 minutos. Esto se debe a que estás consumiendo de un flujo de datos que actualmente está vacío. Ahora enviemos algunos registros al flujo de datos. No cierres esta terminal, mantén el script del consumidor en ejecución y crea una segunda terminal para el siguiente paso.


*Nota*: Para abrir la terminal, haz clic en Terminal -> Nueva Terminal en el menú:

<img src="images/VSCodeCourseraTerminal.png"  width="600"/>

Para abrir una segunda terminal puedes usar el mismo comando. Luego puedes cambiar entre las terminales en el panel inferior derecho:

<img src="images/TerminalsSwitch.png"  width="150"/>


1.4. En la segunda terminal, navegue a la carpeta `src/cli/` y ejecute el script de python `producer_from_cli.py`. El script del productor espera dos argumentos: el nombre del flujo de datos que puede especificar usando la bandera `--stream`, y la cadena JSON que representa el registro del mensaje que se especifica usando la bandera `--json_string`. Para verificar la estructura del registro esperado, puede ver un ejemplo de una cadena JSON en `data/example_record.json` (el mismo ejemplo se usa en el comando proporcionado a continuación). Usted se referirá a esta estructura JSON como la estructura de la carga útil.

```bash
cd src/cli/
python producer_from_cli.py --stream de-c2w2lab1-kinesis-data-stream-cli --json_string '{"session_id": "a1", "customer_number": 100, "city": "Washington", "country": "USA", "credit_limit": 1000, "browse_history": [ {"product_code": "Product1", "quantity": 2, "in_shopping_cart": true}, {"product_code": "Product2", "quantity": 1, "in_shopping_cart": false}]}'
```

Ahora, si va a la primera terminal (la que ejecuta el script del consumidor), debería ver que el consumidor ha leído el registro que acaba de enviar al flujo de datos. ¡Genial! ¡El consumidor está ingiriendo datos en streaming!


1.5. Desde el segundo terminal que ejecuta el script del productor, intenta ahora enviar 4 o 5 registros más al flujo de datos usando la misma estructura para el `--json_string` como se muestra en el paso 1.4. Cambia los atributos de la carga útil para asegurarte de que estás enviando registros diferentes. Recuerda verificar los registros consumidos en el terminal del consumidor.

Una vez que termines de experimentar con el flujo de datos de kinesis, puedes detener el proceso que está ejecutándose en el terminal del consumidor escribiendo `CTRL+C` o `CMD+C`.


<a id='2'></a>
## 2 - Implementando un proceso ETL en streaming


<a id='2-1'></a>
### 2.1 - Creando la Infraestructura para el Proceso ETL de Streaming

**Introducción**

Ahora que tienes una comprensión de los componentes de una plataforma de transmisión de eventos, volvamos a nuestro escenario original de comercio electrónico. Se te proporciona un Kinesis Data Stream que ahora es tu sistema fuente (supón que fue configurado por el ingeniero de software). También se te proporciona el productor que alimenta en el flujo de datos, y tú estás en el lado del consumidor. Necesitarás ingerir datos del flujo de datos, realizar una transformación simple en los datos ingeridos y luego enviar los datos transformados a uno de dos Kinesis Data Streams. Para que puedas explorar los resultados, cada uno de esos flujos de datos será entregado a través de un Kinesis Firehose a su respectido bucket de S3.

<div style="text-align:center"><img src="images/Part2.png"  width="700"/></div>

Comenzarás creando los recursos necesarios usando `boto3` (Dos Kinesis Data Streams, dos instancias de Kinesis Firehose, y dos buckets de S3). Primero entendamos por qué necesitas crear más de un flujo de datos dentro de tu canal de datos.


Crear la infraestructura

Suponga que se le pidió filtrar a los clientes según sus países. Esto se debe a que su empresa notó que los clientes mostraban diferentes comportamientos de compra según sus países. Entonces, si están ubicados en los EE. UU., sus actividades en línea deben ser procesadas por un motor de recomendación determinado. De lo contrario, sus actividades en línea deben ser procesadas por otro motor de recomendación. Para hacerlo, necesita enviar los registros de los clientes de EE. UU. a un flujo de datos diferente de los de otros países, y luego necesita enviar los datos de cada flujo de datos a un bucket diferente para ser procesados de manera diferente por los usuarios posteriores. Esto significa que debe crear un flujo de datos de Kinesis, un Firehose de Kinesis y un bucket de S3 para los clientes de EE. UU. Y luego el mismo conjunto de recursos para clientes internacionales.

¿Cómo sabría la ubicación de los clientes? Los datos que ingresará corresponden al historial de navegación de los usuarios en la página web de comercio electrónico. Estos son los mismos datos con los que trabajó en el laboratorio de la Semana 4 del Curso 1 (la misma estructura que produjo manualmente en la primera parte de este laboratorio) y contienen la ciudad y el país de cada cliente:

```json
{
    "session_id": "a1",
    "customer_number": 100,
    "city": "Washington",
    "country": "USA",
    "credit_limit": 1000,
    "browse_history": [
        {
            "product_code": "Product1",
            "quantity": 2,
            "in_shopping_cart": true
        },
        {
            "product_code": "Product2",
            "quantity": 1,
            "in_shopping_cart": false
        }
    ]
}
```


Sigue las instrucciones para crear los recursos necesarios de acuerdo con los requisitos del negocio.


2.1.1. Ejecuta la siguiente celda para definir dos variables que usarás durante el proceso de creación: tu ID de cuenta de AWS y la región. Trabajarás con `us-east-1` como la región predeterminada como se indica en la segunda variable.


In [ ]:
ACCOUNT_ID = subprocess.run(['aws', 'sts', 'get-caller-identity', '--query', 'Account', '--output', 'text'], capture_output=True, text=True).stdout.strip()
AWS_DEFAULT_REGION = 'us-east-1'

2.1.2. Ahora quieres crear los dos buckets que siguen esta convención de nombres:
- USA: `de-c2w2lab1-{ACCOUNT_ID}-usa`
- Internacional: `de-c2w2lab1-{ACCOUNT_ID}-international`

Para crear los dos buckets, usarás la función `create_s3_bucket()` proporcionada en la siguiente celda, que utiliza las capacidades de `boto3`. Llama a la función dos veces para crear los buckets de USA e internacional. Asegúrate de establecer los dos parámetros bucket_name a los valores de las variables USA_BUCKET y INTERNATIONAL_BUCKET, y de usar `AWS_DEFAULT_REGION` en ambos casos.


In [ ]:
import boto3

USA_BUCKET = f'de-c2w2lab1-{ACCOUNT_ID}-usa'
INTERNATIONAL_BUCKET = f'de-c2w2lab1-{ACCOUNT_ID}-int'


def create_s3_bucket(bucket_name: str, region: str) -> None:
   # Call the boto3 client with the `'s3'` resource and region. 
    s3_client = boto3.client('s3', region_name=region)
    
    # Create the S3 bucket
    try:
        s3_client.create_bucket(Bucket=bucket_name)
        print(f"S3 bucket '{bucket_name}' created successfully in region '{region}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

### START CODE HERE ### (~ 2 lines of code)
# Create the USA bucket
create_s3_bucket(bucket_name=USA_BUCKET, region=AWS_DEFAULT_REGION) # @REPLACE None(bucket_name=None, region=None)
    
# Create the international bucket
create_s3_bucket(bucket_name=INTERNATIONAL_BUCKET, region=AWS_DEFAULT_REGION) # @REPLACE None(bucket_name=None, region=None)
### END CODE HERE ###'

2.1.3. Recuerda de la Semana 1 que puedes usar el comando `aws` para verificar si los buckets existen.


In [ ]:
!aws s3 ls

2.1.4. Ahora, vamos a crear los dos Kinesis Data Streams. Para ello, usarás la función `create_kinesis_data_stream()` proporcionada en la siguiente celda. Llámala para crear los flujos de datos de EE.UU. e Internacional. Ambos deben tener un recuento de fragmentos de 2, lo que significa 2 particiones por flujo, y deben ser nombrados con la siguiente convención:
   - EE.UU.: `de-c2w2lab1-usa-data-stream`
   - Internacional: `de-c2w2lab1-international-data-stream`


In [ ]:
USA_DATA_STREAM = 'de-c2w2lab1-usa-data-stream'
INTERNATIONAL_DATA_STREAM = 'de-c2w2lab1-international-data-stream'

def create_kinesis_data_stream(stream_name: str, shard_count: int = 2) -> None:
    # Call the boto3 client with the `kinesis` resource.  Store the object in `client`.
    client = boto3.client("kinesis")

    # Check if the stream already exists
    if stream_name in client.list_streams()["StreamNames"]:
        print(f"Kinesis data stream {stream_name} already exists")
        return
    
    # Use the `create_stream()` method from the client and pass the data stream name and the shard count.
    response = client.create_stream(StreamName=stream_name, ShardCount=shard_count)
    print("Kinesis data stream created:", response)

### START CODE HERE ### (~ 2 lines of code)
# Create the USA data stream
create_kinesis_data_stream(stream_name=USA_DATA_STREAM, shard_count=2) # @REPLACE None(stream_name=None, shard_count=None)

# Create the International data stream
create_kinesis_data_stream(stream_name=INTERNATIONAL_DATA_STREAM, shard_count=2) # @REPLACE None(stream_name=None, shard_count=None)
### END CODE HERE ###

Usando `boto3`, también puedes verificar el estado de los recursos. Usa la función proporcionada `is_stream_ready()` para ver si tus flujos de datos están listos para ser utilizados.

*Nota*: Si obtienes `false` en la salida aquí, espera unos segundos y vuelve a ejecutar la celda.


In [ ]:
def is_stream_ready(stream_name: str) -> None:
    client = boto3.client("kinesis")
    response = client.describe_stream(StreamName=stream_name)
    return response["StreamDescription"]["StreamStatus"] == "ACTIVE"

# Check if the streams are ready
print(is_stream_ready(stream_name=USA_DATA_STREAM))
print(is_stream_ready(stream_name=INTERNATIONAL_DATA_STREAM))

2.1.5. Finalmente crearás las dos instancias de Kinesis Firehose para las dos ubicaciones, `usa` y `international`. Para ello, llamarás a la siguiente función: `create_kinesis_firehose()` que se proporciona en la siguiente celda. Esta función hace uso del método del cliente `boto3` [create_delivery_stream()](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/firehose/client/create_delivery_stream.html). Si verificas los argumentos pasados a este método, notarás que la fuente y el destino para Kinesis Firehose están configurados (fuente: Kinesis Data Stream y destino: bucket S3). Una vez que estos recursos están configurados, Kinesis Firehose podrá leer automáticamente desde Kinesis Data Stream y escribir automáticamente en el bucket S3. El nombre del rol pasado en la configuración de la fuente y el destino, representa el rol que se adjuntará a Kinesis Firehose para permitirle leer desde un Kinesis Data Stream y escribir en un bucket S3 (nota que `arn` significa nombre de recurso de Amazon y se usa para identificar de manera única los recursos de AWS, puedes aprender más sobre ello [aquí](https://docs.aws.amazon.com/IAM/latest/UserGuide/reference-arns.html#arn-syntax-kinesis-streams)). 

En la siguiente celda, deberás llamar a la función `create_kinesis_firehose()` dos veces para los casos de `usa` e `international`. Necesitas actualizar los parámetros `stream_name` y `bucket_name` con los valores respectivos para USA e internacional en cada una de las llamadas a la función. La función toma otros argumentos que se te proporcionan, como grupos de logs, flujos de logs y el nombre del rol que se adjuntará a Kinesis Firehose:

- Rol IAM de Firehose: `de-c2w2lab1-firehose-role` (Mismo nombre de rol para ambas ubicaciones, lo que permite que la instancia de firehose lea desde un Kinesis Data Stream y escriba en un bucket S3).
- Grupos de logs de CloudWatch: `de-c2w2lab1-firehose-<LOCATION>-log-group`.
- Flujos de logs de CloudWatch: `de-c2w2lab1-<LOCATION>-firehose-log-stream`.

Para los dos últimos puntos, el marcador de posición `<LOCATION>` será reemplazado por `usa` o `international` al pasarlos a la función `create_kinesis_firehose()`.


In [ ]:
def create_kinesis_firehose( firehose_name: str, stream_name: str, bucket_name: str, role_name: str, log_group: str, log_stream:str, account_id: int, region: str):
    # Call the boto3 client with the firehose resource. Assign it to the client variable.
    client = boto3.client("firehose")

    # Check if firehose stream already exists
    if firehose_name in client.list_delivery_streams()["DeliveryStreamNames"]:
        print(f"Kinesis firehose stream {firehose_name} already exists.")
        return
    
    # Use the create_delivery_stream() method of the client object.
    response = client.create_delivery_stream(
        # Pass the firehose name.
        DeliveryStreamName=firehose_name,
        # Specify that the delivery stream uses a Kinesis data stream as a source.
        DeliveryStreamType='KinesisStreamAsSource',
        # Configure the S3 as the destination.
        S3DestinationConfiguration={
            "RoleARN": f"arn:aws:iam::{account_id}:role/{role_name}",
            "BucketARN": f"arn:aws:s3:::{bucket_name}",
            "Prefix": "firehose/",
            "ErrorOutputPrefix": "errors/",
            "BufferingHints": {"SizeInMBs": 1, "IntervalInSeconds": 60},
            "CompressionFormat": "UNCOMPRESSED",  
            "CloudWatchLoggingOptions": {
                "Enabled": True,
                "LogGroupName": log_group, 
                "LogStreamName": log_stream
            },
            "EncryptionConfiguration": {"NoEncryptionConfig": "NoEncryption"},
        },
        # Configure the Kinesis Stream as the Source.
        KinesisStreamSourceConfiguration={
            "KinesisStreamARN": f"arn:aws:kinesis:{region}:{account_id}:stream/{stream_name}",
            "RoleARN": f"arn:aws:iam::{account_id}:role/{role_name}",
        },
    )
    
    print("Kinesis Firehose created:", response)

### START CODE HERE ### (~ 18 lines of code)
# Create the delivery stream for USA orders.
create_kinesis_firehose(firehose_name='de-c2w2lab1-firehose-usa', # @KEEP
                        stream_name=USA_DATA_STREAM, # @REPLACE stream_name=None,
                        bucket_name=USA_BUCKET, # @REPLACE bucket_name=None,
                        role_name='de-c2w2lab1-firehose-role', # @KEEP
                        log_group='de-c2w2lab1-firehose-usa-log-group', # @KEEP
                        log_stream='de-c2w2lab1-usa-firehose-log-stream', # @KEEP
                        account_id=ACCOUNT_ID, # @KEEP
                        region=AWS_DEFAULT_REGION # @KEEP
                       )

# Create the delivery stream for International orders.
create_kinesis_firehose(firehose_name='de-c2w2lab1-firehose-international', # @KEEP
                        stream_name=INTERNATIONAL_DATA_STREAM, # @REPLACE stream_name=None,
                        bucket_name=INTERNATIONAL_BUCKET, # @REPLACE bucket_name=None,
                        role_name='de-c2w2lab1-firehose-role', # @KEEP
                        log_group='de-c2w2lab1-firehose-international-log-group', # @KEEP
                        log_stream='de-c2w2lab1-international-firehose-log-stream', # @KEEP
                        account_id=ACCOUNT_ID, # @KEEP
                        region=AWS_DEFAULT_REGION # @KEEP
                       )
### END CODE HERE ###

<a id='2-2'></a>
### 2.2 - Implementando el ETL en Streaming

Ahora que tus recursos están creados, es momento de que consumas los datos. Para una mejor experiencia de aprendizaje, el productor genera datos dinámicamente con un tiempo medio entre registros de 10 segundos, así que ten eso en cuenta cuando estés consumiendo tus datos y quieras visualizarlos. Durante este consumo, realizarás algunas transformaciones simples sobre los registros antes de enviarlos a las nuevas corrientes de datos creadas con `boto3`. La transformación consistirá en agregar 4 atributos adicionales como se muestra aquí, los cuales serán explicados en los siguientes pasos.

<img src="images/Transformation2.png"  width="800"/>


2.2.1. Ve a la carpeta `src/etl`. Abre el script llamado `consumer.py` (haz doble clic en él). Este archivo es similar al que usaste en la primera parte del laboratorio pero tiene algunas adiciones. Busca la función `poll_shards()`. Este es el lugar donde se leen los datos del Kinesis Data Stream y también donde realizarás algunas transformaciones en cada registro.


2.2.2. Buscar en el ciclo `for record in records_response["Records"]`. La siguiente línea `user_session = json.loads(record["Data"].decode("utf-8"))` carga el registro y lo convierte en un objeto diccionario `user_session` que puedes modificar. El código que está debajo de esta declaración es donde necesitas trabajar en los siguientes pasos.


2.2.3. La primera transformación consiste en agregar la marca de tiempo del procesamiento. Agrega un nuevo atributo al diccionario `user_session`: `"processing_timestamp"` y asigna su valor a la marca de tiempo actual. Usa el paquete `datetime` para ese propósito (`datetime.datetime.now()`).


2.2.4. También se te pide agregar 3 métricas como atributos al diccionario de Python: el número total de productos en el carrito de compras, la cantidad total de productos y el número de productos diferentes en el historial de navegación. Primero, antes del ciclo `for product in user_session["browse_history"]`, crea dos variables `overall_product_quantity` y `overall_in_shopping_cart`. Establécelas en 0. Este ciclo recorre cada uno de los elementos de la lista de historial de navegación.


2.2.5. Contar el número total de cantidades de productos en el historial de navegación. Para ello, tienes que acceder a la clave `"quantity"` de cada producto y sumarla a la variable `overall_product_quantity`. Asegúrate de convertir el valor a un entero antes de sumarlo (usa la función `int()`). Este valor llega como una cadena debido al proceso de decodificación realizado al convertir el registro en un diccionario de Python.


2.2.6. Contar el número de productos en el carrito de compras. Para ello, crea una condición sobre la clave `"in_shopping_cart"` de cada `producto` para contar solo aquellos que están en el carrito de compras. Luego, acumula el valor de `"quantity"` en la variable `overall_in_shopping_cart`. Nuevamente, no olvides convertir el valor de `"quantity"` a un entero antes de sumarlo al valor de `overall_in_shopping_cart`.


2.2.7. Fuera del bucle `for product in user_session["browse_history"]` agrega los valores calculados al diccionario `user_session`: usa las claves `"overall_product_quantity"` y `"overall_in_shopping_cart"` para ese propósito. Además, crea una nueva clave `"total_different_products"` a la que asignarás el número de productos diferentes en el historial de navegación calculando la longitud de la lista de historial de navegación de la sesión del usuario.


2.2.8. Como último paso, crearás un filtro para enviar el registro a la corriente de datos de EE. UU. o a la corriente de datos Internacional. En la misma función `poll_shards()`, puedes ver que tiene `kinesis_dest_stream_names` como un parámetro de entrada, que es un diccionario con dos claves:
 - `"USA"`: El valor contiene el nombre de la corriente de datos donde se colocarán los registros de EE. UU.
 - `"International"`: El valor contiene el nombre de la corriente de datos donde se colocarán los registros Internacionales.
    
En esa función, justo debajo de donde estabas creando tus métricas, encontrarás una llamada al método `kinesis.put_record()`. Para especificar el parámetro `StreamName` para este método, crea un filtro en línea que evalúe la clave `"country"` del diccionario `user_session` y, dependiendo del valor, especifique el nombre de la corriente de datos de destino usando el diccionario `kinesis_dest_stream_names`.

*Nota*: La función hace uso de una [sentencia inline if-else](https://www.geeksforgeeks.org/different-ways-of-using-inline-if-in-python/#basic-inline-using-if-else):
```{python}
StreamName = kinesis_dest_stream_names["USA"] if None["None"] == "USA" else None["International"]
```
Esta sintaxis significa: asignar `kinesis_dest_stream_names["USA"]` a StreamName si la condición `None["None"] == "USA"` es verdadera. De lo contrario, asignar `None["International"]` a StreamName. Por supuesto, en este ejercicio, deberás reemplazar `None` con las palabras clave apropiadas.

Guarda los cambios en el archivo `src/etl/consumer.py`.


2.2.9. Después de esos cambios, en la terminal ve a la carpeta `src/etl` (puede que tengas que usar `cd ../etl` en lugar de `cd src/etl`) y ejecuta el consumidor con el siguiente comando:

```bash
cd src/etl
python consumer.py --source_stream de-c2w2lab1-kinesis-data-stream --dest_streams '{"USA": "de-c2w2lab1-usa-data-stream", "International": "de-c2w2lab1-international-data-stream"}'
```

Al ejecutar este comando, el productor enviará registros al flujo de datos de Kinesis. Este script consumidor leerá esos registros, los transformará y luego los enviará a los flujos de Kinesis apropiados. El Firehose de Kinesis finalmente entregará automáticamente los datos a los buckets de S3.

*Nota*: Este comando se ejecutará continuamente, por lo que mientras no cierres la terminal, los registros seguirán siendo transmitidos.


2.2.10. Para verificar el flujo de destino de EE. UU. o internacional, use otra terminal. Utilice el consumidor de la primera parte del laboratorio ubicado en `src/cli/consumer_from_cli.py` para leer desde el flujo de datos de EE. UU. o internacional y inspeccionar visualmente sus datos transformados.

```bash
cd src/cli/
python consumer_from_cli.py --stream de-c2w2lab1-usa-data-stream
```
Finalmente, puede inspeccionar desde la consola de AWS cada uno de los buckets de S3 para ver cuándo se guardaron los datos (puede encontrar el enlace a la consola de AWS en el paso 1.1). Este proceso puede tardar alrededor de 5-7 minutos en comenzar a ver algún archivo en el bucket de S3 después de que las transformaciones se envían a los flujos de datos.

**¡Felicidades por completar este laboratorio!** Ahora tienes experiencia con la ingesta de streaming usando Kinesis Data Streams.
